# Predicción de datos EEG contaminados con EOG

En este proyecto, se entrena una red neuronal para predecir datos EEG contaminados con EOG utilizando la técnica de eliminación de artefactos utilizando redes neuronales (Feed Forward). Se utilizarán 10 archivos numpy que contienen los datos EEG contaminados con EOG a diferentes niveles de decibelios.

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Se consideran 10 bases de datos con niveles de SNR variados entre -7 dB y 2 dB

In [ ]:
filenames = [f"data/EEG_contaminated_with_EOG/EEG_contaminated_with_EOG_{i}db.npy" for i in range(-7, 3)]

corrs = []

Los datos se contrastaran siempre con la misma base de datos 'x'  pero variando las bases de datos contaminadas en y, posteriormente los datos se dividen en grupos de entrenamiento y validacion de una proporcion 1:10.

El modelo que mejor resultado ha sido con la configuracion de 1 capa oculta (512, 256, 512), iterando por 300 epocas con batches de 32 de tamaño

In [ ]:
for filename in filenames:
    # Cargar los datos
    x = np.load('data/EEG_all_epochs.npy')
    y = np.load(filename)

    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=42)

    # Crear el modelo
    model = Sequential()
    model.add(Dense(512, input_dim=y_train.shape[1], activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='Nadam')

    # Entrenar el modelo
    model.fit(y_train, x_train, epochs=300, batch_size=32, verbose=1)

    # Realizar las predicciones en el conjunto de validación
    y_pred = model.predict(x_val)

    # Calcular el coeficiente de correlación entre las predicciones y los datos de validación limpios
    corr = np.corrcoef(y_val.reshape(-1), y_pred.reshape(-1))[0, 1]
    print(f"Coeficiente de correlación para {filename}: {corr}")
    corrs.append(corr)

Finalmente se evalua la red neuronal con el metodo del coeficiente de correlacion entre la base de datos 'x' y los diferentes 'y' que se testearon, sin embargo, los resultados obtenidos muestran un bajo rendimiento de este tipo de red neuronal, puesto que se testearon diferentes optimizadores, configuraciones y distintos hiperparamtros. Se observa un decenso especialmente alto en cuanto mas sucia esta la señal.

Finalmente se guardan los datos de evaluacion para todos los niveles de contaminacion que se crearon artificialmente en un archivo 'corrs.txt'

In [ ]:
np.savetxt('corrs.txt', corrs)

#TODO: Cambiar activacion sigmoide por activacion relu, actualizar resultados en tabla de conveniencia 
#TODO: Cambiar activacion sigmoide por tanH, actualizar resultados en tabla de conveniencia